In [41]:
%load_ext autoreload
%autoreload 2
import urllib3, json, certifi
import HTMLx
from IPython.display import HTML, display
from operator import itemgetter 

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [42]:
MITREFILE = "enterprise-attack.json"
MITREURL = "https://raw.githubusercontent.com/mitre/cti/master/enterprise-attack/enterprise-attack.json"

In [43]:
http = urllib3.PoolManager(cert_reqs='CERT_REQUIRED',ca_certs=certifi.where())
chunk_size = 1024
r = http.request('GET', MITREURL, preload_content=False)

with open(MITREFILE, 'wb') as out:
    while True:
        data = r.read(chunk_size)
        if not data:
            break
        out.write(data)

r.release_conn()

In [44]:
with open(MITREFILE) as f:
    m = json.load(f)

In [45]:
print( len(m["objects"]))

4548


# Number of objects of different type

In [46]:
res = dict()
for o in m["objects"]:
    #t = o["id"][0:o["id"].find("--")]
    t = o["type"]
    if t == "relationship":
        t += "-"+o["relationship_type"]
    res[t] = res.get(t,0) + 1

display(HTML(HTMLx.html_table([],list(map(lambda x: [x, res[x]],res.keys())))))


In [51]:
platform = "Windows"
display(HTML("<H1>{}</H1>".format(platform)))

## APT actors uses Techniques

In [48]:
res = dict()

for o in m["objects"]:
    if o["type"] == "attack-pattern" and len(list(filter(lambda x: x == platform, o["x_mitre_platforms"]))) > 0:
        
        mitre_t = list(filter(lambda x: x["source_name"] == "mitre-attack", o["external_references"]))[0]["external_id"]
        k = mitre_t + ": " + o["name"]             
        res[k] = dict()
        res[k]["x_mitre_data_sources"] = list()
        res[k]["x_mitre_data_sources"] = o.get("x_mitre_data_sources",[])
          
        for c in ["intrusion-set","malware","tool"]: #APT, #malware, #tools
            res[k][c] = 0
        
        for o2 in m["objects"]: 
            if o2["type"] == "relationship" \
            and o2["relationship_type"] == "uses" \
            and o2["target_ref"] == o["id"]:
                k2 = o2["source_ref"][0:o2["source_ref"].find("--")]
                if k2 in ["intrusion-set","malware","tool"]: 
                    res[k][k2] = res[k].get(k2,0) + 1
                
#print ('%60s  %13s %7s %4s %s' % ("Techiques", "intrusion-set","malware","tool","data_sources"))
#for kk, aa, mm, tt, ds in sorted ( ( (k, res[k]["intrusion-set"], res[k]["malware"], res[k]["tool"],res[k]["x_mitre_data_sources"]) for k in res.keys()), \
#                          key = itemgetter(1), reverse = True ) :
#    print('%60s  %13d %7d %4d %s' % (kk, aa, mm, tt, '\n'.join(ds) ) )

data = list()
for kk, aa, mm, tt, ds in sorted ( ( (k, res[k]["intrusion-set"], res[k]["malware"], res[k]["tool"],res[k]["x_mitre_data_sources"]) for k in res.keys()), \
                          key = itemgetter(1), reverse = True ) :
    data.append([kk, aa, mm, tt, "<BR>".join(ds)])

display(HTML(HTMLx.html_table(["techique", "intrusion-set","malware","tool","data_sources"],data)))

## Telemetry required for Techniques

In [49]:
res = dict()
no_data_sources = list()
for o in m["objects"]:
    t = o["type"]
    if t == "attack-pattern" and len(list(filter(lambda x: x == platform, o["x_mitre_platforms"]))) > 0:
        if o.get("x_mitre_data_sources") is None:
            no_data_sources.append(o["name"])
        else:
            for s in o["x_mitre_data_sources"]:
                res[s] = res.get(s,0) + 1

display(HTML(
    HTMLx.html_table(["Telemetry", "attack-pattern"],\
                     list([key,value] for key, value in sorted(res.items(), key = itemgetter(1), reverse = True)))
))

## Windows techniques without required telemetry specified

In [50]:
for i in no_data_sources:
    print(i)

Port Knocking
